# Complete Guide to Execute Strategy Now

Follow these 6 simple steps to run the code in the above **"FULL PYTHON CODE"** section and view the backtest results instantly. No coding is required — everything has already been set up for you.

## Step 1: Connect to API with Credentials

#### This step enables you to securely log into QuantConnect’s API, before executing your trading agorithm and getting backtest performance results. ####

To break it further down: 
1. It first generates a time-sensitive token by combining your API key with the current time, encrypts it for safety, and formats it so it can be sent over the internet to the Server. 
2. Then, it sends a login request using this token. 
3. Lastly. the Server replies with a result confirming whether login was successful.

The values for **`USER_ID`** and **`API_TOKEN`** used to access API server have been hidden for security reasons. Join QuantConnect Membership today to get your private User ID and API Token — and start executing your own strategy!

In [1]:
from base64 import b64encode ## encodes text in Base64
from hashlib import sha256 ## a blender that turns any text into a scrambled string (used for security)
from time import time ## to get the current time in seconds, used to create a unique timestamp
from requests import get, post ## requests: to send HTTP requests; get: Used to retrieve data from a server; post: Used to send data to a server
import json ## to pretty print JSON reuslts with 4 indentation from API
BASE_URL = 'https://www.quantconnect.com/api/v2/'

# You can find our organization ID at https://www.quantconnect.com/organization/ 
USER_ID = 224901
API_TOKEN = 'b26bdf0c94229806b8515308d77f75b6aa9829d2802540de0c9e9d0dc477da4e'
## ORGANIZATION_ID = '____'

In [ ]:
def get_headers(): ## return the headers needed to send a secure request to the API.
    
    # Get timestamp
    timestamp = f'{int(time())}' ## Gets the current time (like "1722458765"). This makes sure the token is only valid for a short time, adding security.
    time_stamped_token = f'{API_TOKEN}:{timestamp}'.encode('utf-8') ## Combines API token with the current time, separated by a colon, like this:b26bd...da4e:1722458765. Then it turns it into computer-readable bytes using UTF-8.

    # Get hased API token
    hashed_token = sha256(time_stamped_token).hexdigest() ## runs it through the SHA-256 hashing algorithm, producing a long, scrambled string that’s impossible to reverse-engineer.
    authentication = f'{USER_ID}:{hashed_token}'.encode('utf-8') ## Combines User ID and the hashed token, then encodes it into bytes. This is now the login identity in encrypted form.
    authentication = b64encode(authentication).decode('ascii') ## Takes the encoded identity and Base64-encodes it, so it can be safely sent over the internet. Then it decodes it into a readable text string.

    # Create headers dictionary.
    return {
        'Authorization': f'Basic {authentication}', ## your Base64 login string
        'Timestamp': timestamp ## the time you created the token
    } ## This is what you’ll include in your API request to prove who you are.

# Log in to the system to prove your idenify
response = post(f'{BASE_URL}/authenticate', headers = get_headers()) ## 'post' function sends data to API; It sends a request to the URL: www.quantconnect.com/api/v2/authenticate; It includes headers — the login info generated from your get_headers() function.
print("User authenication result:")
print(response.json()) ## response of POST request is saved in variable "response"; shows whether authentication succeeded or not.

# --------------------


# The project ID of the project to manage
project_id = 0 ## to be replaced with the project id created

## Step 2: Create New Project

This step creates a new **`project`** on QuantConnect via API - every algorithm must live inside a project. Consider it nothing more than a package box when you deliver something. 

The **inputs** are: 
1. the project’s name, and 
2. the programming language in use ("Py" for Python). 

It then sends it to the `/projects/create` API endpoint. 

The **output** tells you the new project’s unique ID.


In [49]:
### Create Project
# Send a POST request to the /projects/create endpoint to create a new project
response = post(f'{BASE_URL}/projects/create', headers=get_headers(), json={  ## The json={...} part contains the project information you're sending = a dictionary contains 2 items: Project Name and Language. 
    "name": f"Project_{int(time())}",  # Unique project name using current timestamp
    "language": "Py"  # Programming language for the project (Python)
})

# Parse the JSON response into python managable dict
result = response.json() ## json() converts JSON response to distionary - not the other way around

# Extract the project ID from the response
project_id = result['projects'][0]['projectId'] ## "result" is a nested disctionary: first level got 2 keys: 'success' and 'projects', then under 'projects', it contains nested 2 disctionaries with keys: 'projectId' and 'name' 
## [0] here access the first nested dictionary with key = 'projectId'

# Check if the request was successful and print the result
if result['success']:
    print("Project Created Successfully. Your Project ID is " + str(project_id) + ".\n")
    ## print(result)
    print(json.dumps(result, indent=4))

Project Created Successfully. Your Project ID is 23808805.
{
    "projects": [
        {
            "projectId": 23808805,
            "organizationId": "0aa48a67d71e36c6a3870ebb8c224664",
            "name": "Project_1751528141",
            "modified": "2025-07-03 07:35:44",
            "created": "2025-07-03 07:35:44",
            "ownerId": 224901,
            "language": "Py",
            "collaborators": [
                {
                    "uid": 224901,
                    "liveControl": true,
                    "permission": "write",
                    "publicId": "haixiang-yan",
                    "profileImage": "https://cdn.quantconnect.com/web/i/users/profile/4f9b4ae886ebdb31674640077.png",
                    "email": "hxyan.2015@gmail.com",
                    "name": "Haixiang Yan",
                    "bio": null,
                    "owner": true
                }
            ],
            "leanVersionId": 17170,
            "leanPinnedToMaster": true,
       

## Step 3: Create Algorithm File

This step puts the **`algorithm file`** inside the QuantConnect project you defined in step 2 (the "package box") via API.

The **inputs** are: 
1. the file name you defined (e.g. `utils.py`),
2. its content (the trading algorithm), and 
3. the associated project ID (the "package box"). 

It then sends these to QuantConnect’s `/files/create` endpoint. 

The **output** is a response confirming success and returning the uploaded file name.


In [50]:
### Create File
import json
import os

# Define relative path to the algorithm file
file_path = os.path.join("algos", "algo_0504.py")

# Read the code content from the subfolder
with open(file_path, "r", encoding="utf-8") as file:
    code_content = file.read()

# Send a POST request to the /files/create endpoint to create a new file
response = post(f'{BASE_URL}/files/create', headers=get_headers(), json={
    "projectId": project_id,  # ID of the project
    "name": "algo_0504.py",  ## Name of the algorithm file
    "content": code_content  # Content of the algorithm file - contains exactly thw same algorithm as defined under "FULL PYTHON CODE section"
})

# Parse the JSON response into python managable dict
result = response.json()
## print(result)

# Check if the request was successful and print the result
if result['success']:
    ## file_name = result.get("files", {}).get("name", "Unknown File")
    files = result.get("files", [])
    if files and isinstance(files, list):
        file_name = files[0].get("name", "Unknown File")
    else:
        file_name = "Unknown File"

    print("Algorithm Created Successfully. Your Algorithm Name is " + file_name + ".\n")
    ## print(result)
    print(json.dumps(result, indent=4))

File Created Successfully. Your File Name is utils.py
{
    "files": [
        {
            "id": 95975133,
            "name": "utils.py",
            "content": "#!/usr/bin/env python\n# coding: utf-8\n\n# In[ ]:\n\n\nfrom AlgorithmImports import *\n\nclass JanuaryEffectInStocks(QCAlgorithm):\n\n    def Initialize(self):\n        self.SetStartDate(2000, 1, 1)  \n        self.SetCash(100000) \n\n        data = self.AddEquity(\"SPY\", Resolution.Daily)\n        data.SetLeverage(10)\n        self.large_cap = data.Symbol\n        \n        data = self.AddEquity(\"IWM\", Resolution.Daily)\n        data.SetLeverage(10)\n        self.small_cap = data.Symbol\n\n        self.start_price = None\n        self.recent_month = -1\n        \n    def OnData(self, data):\n        if self.recent_month == self.Time.month:\n            return\n        self.recent_month = self.Time.month\n\n        if self.Securities[self.large_cap].GetLastData() and self.Securities[self.small_cap].GetLastData():\n     

## Step 4: Create Compilation Job

This step sends a request to **compile** the uploaded algorithm file on QuantConnect. 

The **input** is the `projectId`, telling the system which project to compile. 

The **output** confirms success and prints the `compileId`, showing your code has been successfully compiled and is ready to run for backtest.


In [53]:
### Create Compilation Job
# Prepare data payload to create a compilation job
payload = {
    "projectId": project_id  # ID of the project to compile
}

# Send a POST request to the /compile/create endpoint to start compilation
response = post(f'{BASE_URL}/compile/create', headers=get_headers(), json=payload)

# Parse the JSON response into python managable dict
result = response.json()

# Extract the compile ID from the response
compile_id = result['compileId']

# Check if the request was successful and print the result
if result['success']:
    print("Compilation Job Created Successfully for Your Algorithm. Your Compliance ID is " + str(compile_id) + ".\n")
    print(result)
    ## print(result)
    print(json.dumps(result, indent=4))

Compilation Job Created Successfully. Your Compliance ID is 7b8db58de8683dcf75d232de3cbe457b-8321e6cf7a315a90351d2e46ffb84f94.

{'compileId': '7b8db58de8683dcf75d232de3cbe457b-8321e6cf7a315a90351d2e46ffb84f94', 'state': 'InQueue', 'parameters': [], 'projectId': '23808805', 'signature': '8321e6cf7a315a90351d2e46ffb84f94', 'signatureOrder': ['project/main.py', 'project/utils.py'], 'success': True}
{
    "compileId": "7b8db58de8683dcf75d232de3cbe457b-8321e6cf7a315a90351d2e46ffb84f94",
    "state": "InQueue",
    "parameters": [],
    "projectId": "23808805",
    "signature": "8321e6cf7a315a90351d2e46ffb84f94",
    "signatureOrder": [
        "project/main.py",
        "project/utils.py"
    ],
    "success": true
}


## Step 5: Start Backtesting

This step creates a **backtest** for your trading algorithm on QuantConnect. 

The **input** includes: 
1. your `projectId`, 
2. a successful `compileId`, and 
3. a backtest name. 

It then sends these inputs to the `/backtests/create` endpoint. 

The **output** is a confirmation with a unique `backtestId`, showing your strategy is now running historically to test performance over past market data. Please note that depends on how complex your algorithm is, it may take from few seconds to few minutes to complete the process. 

In [56]:
### Create Backtest
# Define compilation ID (the ones have been run successfully)
## compile_id = "compile_id..."

# Send a POST request to the /backtests/create endpoint to create a backtest
response = post(f'{BASE_URL}/backtests/create', headers=get_headers(), json={
    "projectId": project_id,  # ID of the project to backtest
    "compileId": compile_id,  # Compilation ID for the backtest
    "backtestName": f"Backtest {int(time())}"  # Unique name for the backtest using current timestamp
})

# Parse the JSON response into python managable dict
result = response.json()

# Extract the backtest ID from the response
backtest_id = result['backtest']['backtestId']

# Check if the request was successful and print the result
if result['success']:
    print("Hurray! You completed the backtest successfully. Your Backtest ID is " + str(backtest_id) + ".\n")
    ## print(result)
    print(json.dumps(result, indent=4))
    print(backtest_id)

Hurray! You completed the backtest successfully. Your Backtest ID is f80fd71310cee28a305bb66c9bfd5e96.

{
    "backtest": {
        "note": null,
        "name": "Backtest 1751529017",
        "organizationId": 218385,
        "projectId": 23808805,
        "completed": false,
        "optimizationId": null,
        "backtestId": "f80fd71310cee28a305bb66c9bfd5e96",
        "tradeableDates": 0,
        "researchGuide": {
            "minutes": 1,
            "backtestCount": 3,
            "parameters": 11
        },
        "backtestStart": "2025-07-03 07:50:20",
        "backtestEnd": "2025-07-03 07:50:20",
        "created": "2025-07-03 07:50:20",
        "snapshotId": 23808962,
        "status": "In Queue...",
        "error": null,
        "stacktrace": null,
        "progress": 0,
        "hasInitializeError": false,
        "charts": {
            "Strategy Equity": {
                "name": "Strategy Equity"
            },
            "Benchmark": {
                "name": "Benc

## Step 6: Read Backtest Performance

This final step retrieves and displays performance results of a completed backtest on QuantConnect. 

The **inputs** include `projectId` and `backtestId`. 

It then sends these to the `/backtests/read` endpoint. 

The **output** is a detailed table showing metrics like Sharpe Ratio, annual return, drawdown, and profit. These results help evaluate how well your trading strategy performed.

In [67]:
### Read Backtest Statistics
## Might need to wait few mins before getting results
# Prepare data payload to read backtest performance
import json
from tabulate import tabulate

payload = {
    "projectId": project_id,  # ID of the project
    "backtestId": backtest_id  # ID of the backtest to read
}

# Send a POST request to the /backtests/read endpoint to get statistics
response = post(f'{BASE_URL}/backtests/read', headers=get_headers(), json=payload)

# Parse the JSON response into python managable dict
result = response.json()
##print(result.get("success",{}))
# Check if the request was successful and print the statistics   
if result['success']:
    print("Backtest Performance:")
    backtestInfo = result.get("backtest",{})
    stats = result.get("backtest",{}).get("totalPerformance",{}).get("portfolioStatistics", {})
    ## print(stats)
    metrics = {
        "Trading Days": backtestInfo.get("tradeableDates", "N/A"),
        "Algo Start Time": backtestInfo.get("backtestStart", "N/A"),
        "Algo End Time": backtestInfo.get("backtestEnd", "N/A"),
        "Current Asset Balance": (
            f"{float(stats.get('endEquity')):.2f}"
            if stats.get("endEquity") not in [None, "N/A"]
            else "N/A"
        ),
        "Compounding Annual Return": (
            f"{float(stats.get('compoundingAnnualReturn')) * 100:.2f}%"
            if stats.get('compoundingAnnualReturn') not in [None, "N/A"]
            else "N/A"
        ),
        "Drawdown": (
            f"{float(stats.get('drawdown')) * 100:.2f}%"
            if stats.get('drawdown') not in [None, "N/A"]
            else "N/A"
        ),
        "Total Net Profit": (
            f"{float(stats.get('totalNetProfit')) * 100:.2f}%"
            if stats.get('totalNetProfit') not in [None, "N/A"]
            else "N/A"
        ),
        "Sharpe Ratio": (
            f"{float(stats.get('sharpeRatio')):.2f}"
            if stats.get("sharpeRatio") not in [None, "N/A"]
            else "N/A"
        ),
        "Sortino Ratio": (
            f"{float(stats.get('sortinoRatio')):.2f}"
            if stats.get("sortinoRatio") not in [None, "N/A"]
            else "N/A"
        ),
        "Treynor Ratio": (
            f"{float(stats.get('treynorRatio')):.2f}"
            if stats.get("treynorRatio") not in [None, "N/A"]
            else "N/A"
        ),
        "Beta": (
            f"{float(stats.get('beta')):.2f}"
            if stats.get("beta") not in [None, "N/A"]
            else "N/A"
        ),
        "Information Ratio": (
            f"{float(stats.get('informationRatio')):.2f}"
            if stats.get("informationRatio") not in [None, "N/A"]
            else "N/A"
        ),
        "Tracking Error": (
            f"{float(stats.get('trackingError')) * 100:.2f}%"
            if stats.get('trackingError') not in [None, "N/A"]
            else "N/A"
        ),
        "Portfolio Turnover": (
            f"{float(stats.get('portfolioTurnover')) * 100:.2f}%"
            if stats.get('portfolioTurnover') not in [None, "N/A"]
            else "N/A"
        ),
        "VaR 99": stats.get("valueAtRisk99", "N/A"),
        "VaR 95": stats.get("valueAtRisk95", "N/A")
    }

    print(tabulate(metrics.items(), headers=["Metric", "Value"], tablefmt="grid"))

# print the entire backtest statistics

## if result['success']:
    ## print("Backtest Statistics:")
    ### print(result)
    ## print(json.dumps(result, indent=4)) 

Backtest Performance:
+---------------------------+---------------------+
| Metric                    | Value               |
+===========================+=====================+
| Trading Days              | 376                 |
+---------------------------+---------------------+
| Algo Start Time           | 2024-01-02 00:00:00 |
+---------------------------+---------------------+
| Algo End Time             | 2025-07-02 23:59:59 |
+---------------------------+---------------------+
| Current Asset Balance     | 117412.83           |
+---------------------------+---------------------+
| Compounding Annual Return | 11.29%              |
+---------------------------+---------------------+
| Drawdown                  | 17.90%              |
+---------------------------+---------------------+
| Total Net Profit          | 17.41%              |
+---------------------------+---------------------+
| Sharpe Ratio              | 0.26                |
+---------------------------+-------------

### Congratulations! You've successfully executed your strategy and retrieved the performance metrics!